In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
import sys
sys.path.append('../..')
from modules.many_features import utils, constants
import matplotlib.pyplot as plt
%matplotlib inline

#### The data used

In [4]:
# #df = pd.read_csv('data/anemia_synth_dataset_hb_some_nans.csv')
# df = pd.read_csv('data/anemia_synth_dataset_with_unspecified.csv')
# df = df.fillna(0)
# # classes = list(df.label.unique())
# # nums = [i for i in range(len(classes))]
# # class_dict = dict(zip(classes, nums))
# class_dict = {'No anemia': 0, 'Hemolytic anemia': 1, 'Aplastic anemia': 2, 'Iron deficiency anemia': 3,
#               'Vitamin B12/Folate deficiency anemia': 4, 'Anemia of chronic disease': 5, 'Unspecified anemia': 6}
# df['label'] = df['label'].replace(class_dict)
# X = df.iloc[:, 0:-1]
# y = df.iloc[:, -1]

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
# X_train, y_train = np.array(X_train), np.array(y_train)
# X_test, y_test = np.array(X_test), np.array(y_test)
# X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [13]:
test_set = pd.read_csv('../../data/more_features/train_sets/test_set_constant.csv')
X_test = test_set.iloc[:, 0:-1]
y_test = test_set.iloc[:, -1]
X_test, y_test = np.array(X_test), np.array(y_test)
X_test.shape, y_test.shape

((14000, 17), (14000,))

In [14]:
test_df =pd.read_csv('../../test_dfs/many_features/0.1/correlated/constant_test_set/test_df_noisy6_17000000.csv')
test_df.head()

,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred
0,4.0,0.0,1.0,1.0,0.0,"['hemoglobin', 'mcv', 'ret_count', 'Aplastic a...",6.0,6.0
1,4.0,1.0,1.0,1.0,0.0,"['hemoglobin', 'mcv', 'segmented_neutrophils',...",2.0,2.0
2,4.0,2.0,1.0,1.0,0.0,"['hemoglobin', 'mcv', 'segmented_neutrophils',...",2.0,2.0
3,4.0,3.0,0.0,-1.0,1.0,"['hemoglobin', 'mcv', 'segmented_neutrophils',...",5.0,7.0
4,4.0,4.0,1.0,1.0,0.0,"['hemoglobin', 'mcv', 'ret_count', 'Hemolytic ...",5.0,5.0


In [15]:
test_df.y_pred.isna().sum()/len(test_df)*100

0.0

In [16]:
def get_min_of_array(array, n):
    '''The n least elements in an array'''
    idx = np.argpartition(array, n)
    min_vals = array[idx[:n]]
    return min_vals

In [17]:
X_test_df = pd.concat([pd.DataFrame(X_test), pd.DataFrame(y_test)], axis=1)
X_test_df.columns = df.columns
X_test_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,11.391136,-1.000000,1.504298,5.058881,467.197112,96.252926,217.860499,3.550376,1.0,-1.000000,36.703810,122.310168,49.897027,20.353251,-1.000000,34.173407,46.631388,6
1,10.355048,272.506966,-1.000000,0.000000,128.706486,101.912313,-1.000000,3.048223,1.0,-1.000000,-1.000000,-1.000000,29.327349,-1.000000,-1.000000,31.065144,-1.000000,2
2,11.159385,-1.000000,-1.000000,0.000000,-1.000000,103.395616,77.212369,3.237870,0.0,-1.000000,8.690122,65.554731,19.167967,7.239049,-1.000000,33.478155,-1.000000,2
3,8.179735,140.876632,5.484515,-1.000000,303.740826,99.877458,155.573175,2.456931,0.0,-1.000000,41.123526,119.433840,39.384848,6.394235,-1.000000,24.539204,51.219053,5
4,9.916825,-1.000000,3.909810,-1.000000,-1.000000,90.543986,189.347916,3.285748,0.0,1.501433,34.905667,59.177001,58.538524,22.843594,139.245204,29.750475,-1.000000,5


In [18]:
a1 = np.array(test_df['y_actual'], dtype=np.float32)
a2 = np.array(X_test_df['label'], dtype=np.float32)
assert np.array_equal(a1, a2)

#### 0 - No Anemia

In [19]:
no_test_df = test_df[test_df.y_actual== 0]
no_x_test_df = X_test_df[X_test_df.label==0]
combined_no_test_df = pd.concat([no_test_df, no_x_test_df], axis=1)
combined_no_test_df.head()

,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred,hemoglobin,ferritin,...,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
6,3.0,6.0,1.0,1.0,0.0,"['hemoglobin', 'gender', 'No anemia']",0.0,0.0,12.705878,466.949486,...,0.0,-1.000000,15.216632,54.333997,9.541122,13.394647,-1.000000,38.117635,95.897797,0
12,3.0,12.0,1.0,1.0,0.0,"['hemoglobin', 'gender', 'No anemia']",0.0,0.0,14.015774,-1.000000,...,0.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,42.047321,-1.000000,0
14,2.0,14.0,1.0,1.0,0.0,"['hemoglobin', 'No anemia']",0.0,0.0,13.116419,-1.000000,...,1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,39.349256,-1.000000,0
17,3.0,17.0,1.0,1.0,0.0,"['hemoglobin', 'gender', 'No anemia']",0.0,0.0,16.980507,239.806877,...,1.0,1.432289,46.354935,67.460446,47.054099,24.324662,98.555978,50.941521,133.686509,0
21,3.0,21.0,1.0,1.0,0.0,"['hemoglobin', 'gender', 'No anemia']",0.0,0.0,12.229142,224.718388,...,0.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,36.687427,-1.000000,0


In [20]:
combined_no_test_df.y_pred.value_counts()

0.0    1997
7.0       2
5.0       1
Name: y_pred, dtype: int64

In [26]:
df_0 = combined_no_test_df[combined_no_test_df.y_pred==5]
df_0

,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred,hemoglobin,ferritin,...,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
13223,5.0,13223.0,0.0,-1.0,0.0,"['hemoglobin', 'gender', 'mcv', 'ret_count', '...",0.0,5.0,13.005926,9.70778,...,1.0,-1.0,11.722058,44.516766,6.51418,29.66895,-1.0,39.017777,21.5814,0


In [27]:
df_0.describe()[['hemoglobin', 'gender', 'mcv', 'ret_count']]

,hemoglobin,gender,mcv,ret_count
count,1.000000,1.0,1.000000,1.000000
mean,13.005926,1.0,90.843476,3.734518
std,NaN,NaN,NaN,NaN
min,13.005926,1.0,90.843476,3.734518
25%,13.005926,1.0,90.843476,3.734518
50%,13.005926,1.0,90.843476,3.734518
75%,13.005926,1.0,90.843476,3.734518
max,13.005926,1.0,90.843476,3.734518


#### 1 - Vitamin b12

In [37]:
vit_test_df = test_df[test_df.y_actual == 1]
vit_x_test_df = X_test_df[X_test_df.label== 1]
combined_vit_test_df = pd.concat([vit_test_df, vit_x_test_df], axis=1)
combined_vit_test_df.head()

,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred,hemoglobin,ferritin,...,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
15,5.0,15.0,1.0,1.0,0.0,"['hemoglobin', 'gender', 'mcv', 'segmented_neu...",1.0,1.0,12.317586,258.583177,...,1.0,-1.000000,129.701870,98.960747,73.848869,2.476396,-1.000000,36.952757,-1.000000,1
25,4.0,25.0,1.0,1.0,0.0,"['hemoglobin', 'mcv', 'segmented_neutrophils',...",1.0,1.0,10.549702,-1.000000,...,0.0,-1.000000,-1.000000,-1.000000,76.999349,-1.000000,-1.000000,31.649106,-1.000000,1
29,4.0,29.0,1.0,1.0,0.0,"['hemoglobin', 'mcv', 'segmented_neutrophils',...",1.0,1.0,8.513382,63.670459,...,1.0,-1.000000,-1.000000,-1.000000,10.907405,-1.000000,-1.000000,25.540146,-1.000000,1
30,5.0,30.0,1.0,1.0,0.0,"['hemoglobin', 'gender', 'mcv', 'segmented_neu...",1.0,1.0,12.749833,199.961561,...,1.0,-1.000000,-1.000000,-1.000000,4.610539,-1.000000,-1.000000,38.249498,-1.000000,1
33,4.0,33.0,1.0,1.0,0.0,"['hemoglobin', 'mcv', 'segmented_neutrophils',...",1.0,1.0,11.068506,3.952179,...,0.0,1.703951,120.079339,108.801489,75.748430,8.198632,106.618531,33.205519,48.787061,1


In [38]:
combined_vit_test_df.y_pred.value_counts()

1.0    1788
7.0       6
2.0       5
0.0       1
Name: y_pred, dtype: int64

In [45]:
df_1 = combined_vit_test_df[combined_vit_test_df.y_pred==0]
df_1.head()

,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred,hemoglobin,ferritin,...,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
7174,3.0,7174.0,0.0,-1.0,0.0,"['hemoglobin', 'gender', 'No anemia']",1.0,0.0,11.995296,233.540098,...,0.0,-1.0,-1.0,-1.0,49.18609,-1.0,-1.0,35.985887,-1.0,1


In [46]:
df_1.describe()[['hemoglobin', 'gender', 'mcv', 'segmented_neutrophils']]

,hemoglobin,gender,mcv,segmented_neutrophils
count,1.000000,1.0,1.000000,1.000000
mean,11.995296,0.0,100.809763,5.298547
std,NaN,NaN,NaN,NaN
min,11.995296,0.0,100.809763,5.298547
25%,11.995296,0.0,100.809763,5.298547
50%,11.995296,0.0,100.809763,5.298547
75%,11.995296,0.0,100.809763,5.298547
max,11.995296,0.0,100.809763,5.298547


In [47]:
# a4= vit_test_df[vit_test_df.y_actual != vit_test_df.y_pred]
# a4.head()

#### 2 - Unspecified Anemia

In [48]:
unspec_test_df = test_df[test_df.y_actual == 2]
unspec_x_test_df = X_test_df[X_test_df.label== 2]
combined_unspec_test_df = pd.concat([unspec_test_df, unspec_x_test_df], axis=1)
combined_unspec_test_df.head()

,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred,hemoglobin,ferritin,...,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
1,4.0,1.0,1.0,1.0,0.0,"['hemoglobin', 'mcv', 'segmented_neutrophils',...",2.0,2.0,10.355048,272.506966,...,1.0,-1.000000,-1.000000,-1.000000,29.327349,-1.000000,-1.000000,31.065144,-1.0,2
2,4.0,2.0,1.0,1.0,0.0,"['hemoglobin', 'mcv', 'segmented_neutrophils',...",2.0,2.0,11.159385,-1.000000,...,0.0,-1.000000,8.690122,65.554731,19.167967,7.239049,-1.000000,33.478155,-1.0,2
13,4.0,13.0,1.0,1.0,0.0,"['hemoglobin', 'mcv', 'segmented_neutrophils',...",2.0,2.0,7.072613,-1.000000,...,1.0,1.949093,48.359508,32.509684,-1.000000,25.465037,-1.000000,21.217840,-1.0,2
24,4.0,24.0,1.0,1.0,0.0,"['hemoglobin', 'mcv', 'segmented_neutrophils',...",2.0,2.0,11.053152,-1.000000,...,0.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,33.159457,-1.0,2
42,4.0,42.0,1.0,1.0,0.0,"['hemoglobin', 'mcv', 'segmented_neutrophils',...",2.0,2.0,6.485627,-1.000000,...,1.0,0.619290,86.723720,45.400427,-1.000000,20.661100,107.563644,19.456882,-1.0,2


In [49]:
combined_unspec_test_df.y_pred.value_counts()

2.0    1804
0.0       3
Name: y_pred, dtype: int64

In [50]:
df_2 = combined_unspec_test_df[combined_unspec_test_df.y_pred==0]
df_2.head()

,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred,hemoglobin,ferritin,...,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
4850,3.0,4850.0,0.0,-1.0,0.0,"['hemoglobin', 'gender', 'No anemia']",2.0,0.0,11.996847,170.077558,...,0.0,-1.000000,-1.000000,-1.000000,64.835594,-1.000000,-1.000000,35.990541,-1.000000,2
7218,3.0,7218.0,0.0,-1.0,0.0,"['hemoglobin', 'gender', 'No anemia']",2.0,0.0,11.995926,129.237232,...,0.0,1.692699,19.331969,94.734681,51.577199,1.934564,43.242117,35.987778,-1.000000,2
11868,3.0,11868.0,0.0,-1.0,0.0,"['hemoglobin', 'gender', 'No anemia']",2.0,0.0,11.994458,295.893125,...,0.0,-1.000000,11.448570,74.537399,75.453941,2.058594,-1.000000,35.983375,24.001207,2


In [51]:
df_2.describe()[['hemoglobin', 'gender', 'mcv', 'segmented_neutrophils']]

,hemoglobin,gender,mcv,segmented_neutrophils
count,3.000000,3.0,3.000000,3.0
mean,11.995744,0.0,101.911575,0.0
std,0.001205,0.0,1.364702,0.0
min,11.994458,0.0,100.384769,0.0
25%,11.995192,0.0,101.361011,0.0
50%,11.995926,0.0,102.337253,0.0
75%,11.996386,0.0,102.674978,0.0
max,11.996847,0.0,103.012703,0.0


#### 5 - Hemolytic Anemia

In [58]:
hemolytic_test_df = test_df[test_df.y_actual == 5]
hemolytic_x_test_df = X_test_df[X_test_df.label== 5]
combined_hemolytic_test_df = pd.concat([hemolytic_test_df, hemolytic_x_test_df], axis=1)
combined_hemolytic_test_df.head()

,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred,hemoglobin,ferritin,...,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
3,4.0,3.0,0.0,-1.0,1.0,"['hemoglobin', 'mcv', 'segmented_neutrophils',...",5.0,7.0,8.179735,140.876632,...,0.0,-1.000000,41.123526,119.433840,39.384848,6.394235,-1.000000,24.539204,51.219053,5
4,4.0,4.0,1.0,1.0,0.0,"['hemoglobin', 'mcv', 'ret_count', 'Hemolytic ...",5.0,5.0,9.916825,-1.000000,...,0.0,1.501433,34.905667,59.177001,58.538524,22.843594,139.245204,29.750475,-1.000000,5
7,4.0,7.0,1.0,1.0,0.0,"['hemoglobin', 'mcv', 'ret_count', 'Hemolytic ...",5.0,5.0,9.170892,-1.000000,...,0.0,0.887628,137.161446,57.501898,14.628888,18.963912,-1.000000,27.512676,-1.000000,5
20,4.0,20.0,1.0,1.0,0.0,"['hemoglobin', 'mcv', 'ret_count', 'Hemolytic ...",5.0,5.0,6.713676,182.992976,...,0.0,0.976512,123.572402,115.091996,77.437026,27.174828,60.655513,20.141028,85.369996,5
26,4.0,26.0,1.0,1.0,0.0,"['hemoglobin', 'mcv', 'ret_count', 'Hemolytic ...",5.0,5.0,6.036641,-1.000000,...,0.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,18.109922,-1.000000,5


In [60]:
combined_hemolytic_test_df.y_pred.value_counts()

5.0    1591
7.0     123
3.0      26
1.0      25
6.0      20
4.0       7
0.0       3
Name: y_pred, dtype: int64

In [61]:
df_5 = combined_hemolytic_test_df[combined_hemolytic_test_df.y_pred==7]
df_5.head()

,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred,hemoglobin,ferritin,...,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
3,4.0,3.0,0.0,-1.0,1.0,"['hemoglobin', 'mcv', 'segmented_neutrophils',...",5.0,7.0,8.179735,140.876632,...,0.0,-1.000000,41.123526,119.433840,39.384848,6.394235,-1.000000,24.539204,51.219053,5
82,4.0,82.0,0.0,-1.0,1.0,"['hemoglobin', 'mcv', 'tibc', 'Inconclusive di...",5.0,7.0,10.088944,-1.000000,...,1.0,-1.000000,111.680921,52.805488,68.385836,4.737230,83.039059,30.266833,-1.000000,5
123,4.0,123.0,0.0,-1.0,1.0,"['hemoglobin', 'mcv', 'segmented_neutrophils',...",5.0,7.0,9.239713,-1.000000,...,0.0,1.217293,92.395193,61.875482,45.406923,23.405829,62.485530,27.719139,-1.000000,5
159,4.0,159.0,0.0,-1.0,1.0,"['hemoglobin', 'mcv', 'tibc', 'Inconclusive di...",5.0,7.0,9.387371,-1.000000,...,1.0,0.302836,5.903959,106.339055,-1.000000,0.830222,-1.000000,28.162113,-1.000000,5
223,4.0,223.0,0.0,-1.0,1.0,"['hemoglobin', 'mcv', 'segmented_neutrophils',...",5.0,7.0,9.029745,428.099026,...,1.0,-1.000000,90.079120,47.388689,42.571897,6.479223,-1.000000,27.089235,38.986070,5


In [63]:
df_5.describe()[['hemoglobin', 'gender', 'mcv', 'ret_count']]

,hemoglobin,gender,mcv,ret_count
count,123.000000,123.000000,123.000000,123.000000
mean,9.540723,0.552846,90.403774,4.067323
std,1.834465,0.499233,9.390636,1.138376
min,6.018169,0.000000,80.050155,2.126087
25%,8.242543,0.000000,80.602693,3.200758
50%,9.387371,1.000000,98.839114,3.989140
75%,11.040210,1.000000,99.334055,5.084179
max,12.944645,1.000000,99.945852,5.974165


In [55]:
a1_index_list = list(a1.index)
a1_x_test_df = X_test_df.iloc[a1_index_list]
len(a1_x_test_df)

62

In [60]:
a1_x_test_df.describe()

,ferritin,ret_count,segmented_neutrophils,iron,tibc,rbc,mcv,mentzer_index,label
count,62.0,62.000000,62.0,62.0,62.0,62.000000,62.000000,62.000000,62.0
mean,0.0,1.901234,0.0,0.0,0.0,3.179949,88.090720,29.816013,2.0
std,0.0,0.351053,0.0,0.0,0.0,0.817971,4.156499,8.790087,0.0
min,0.0,0.006229,0.0,0.0,0.0,1.696806,80.602310,15.525618,2.0
25%,0.0,1.952360,0.0,0.0,0.0,2.483316,85.103664,23.066521,2.0
50%,0.0,1.973896,0.0,0.0,0.0,3.132505,88.698947,28.316785,2.0
75%,0.0,1.987303,0.0,0.0,0.0,3.786011,90.663813,35.495148,2.0
max,0.0,1.999754,0.0,0.0,0.0,5.191569,98.037468,54.087862,2.0


#### 3 - Anemia of chronic disease

In [52]:
acd_test_df = test_df[test_df.y_actual == 3]
acd_x_test_df = X_test_df[X_test_df.label== 3]
combined_acd_test_df = pd.concat([acd_test_df, acd_x_test_df], axis=1)
combined_acd_test_df.head()

,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred,hemoglobin,ferritin,...,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
10,4.0,10.0,1.0,1.0,0.0,"['hemoglobin', 'mcv', 'tibc', 'Anemia of chron...",3.0,3.0,7.524126,-1.000000,...,0.0,1.045661,2.969448,81.203661,-1.000000,12.533430,88.951873,22.572378,47.655859,3
28,4.0,28.0,1.0,1.0,0.0,"['hemoglobin', 'mcv', 'tibc', 'Anemia of chron...",3.0,3.0,10.406249,252.559558,...,1.0,1.268623,57.160148,94.192545,-1.000000,23.593935,63.199066,31.218748,128.462979,3
37,6.0,37.0,1.0,1.0,0.0,"['hemoglobin', 'mcv', 'tibc', 'ferritin', 'hem...",3.0,3.0,6.702709,108.376181,...,0.0,-1.000000,13.115453,116.374708,71.812986,23.539670,-1.000000,20.108128,12.687201,3
47,4.0,47.0,1.0,1.0,0.0,"['hemoglobin', 'mcv', 'tibc', 'Anemia of chron...",3.0,3.0,7.698529,474.944412,...,1.0,-1.000000,-1.000000,-1.000000,10.180058,-1.000000,-1.000000,23.095587,-1.000000,3
61,4.0,61.0,1.0,1.0,0.0,"['hemoglobin', 'mcv', 'tibc', 'Anemia of chron...",3.0,3.0,6.885305,-1.000000,...,0.0,0.929479,66.668084,117.051851,48.305638,25.341432,113.589985,20.655914,23.863025,3


In [53]:
combined_acd_test_df.y_pred.value_counts()

3.0    1702
7.0     177
4.0      68
6.0       2
0.0       2
Name: y_pred, dtype: int64

In [56]:
df_3 = combined_acd_test_df[combined_acd_test_df.y_pred==7]
df_3.head()

,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred,hemoglobin,ferritin,...,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
135,6.0,135.0,0.0,-1.0,1.0,"['hemoglobin', 'mcv', 'tibc', 'gender', 'ferri...",3.0,7.0,7.528958,-1.000000,...,1.0,-1.000000,11.767728,98.753028,61.017693,21.792071,-1.000000,22.586874,26.988202,3
138,4.0,138.0,0.0,-1.0,1.0,"['hemoglobin', 'mcv', 'tibc', 'Inconclusive di...",3.0,7.0,11.238644,327.687083,...,0.0,1.915884,77.287296,45.173135,63.997943,14.314223,98.159105,33.715933,-1.000000,3
140,4.0,140.0,0.0,-1.0,1.0,"['hemoglobin', 'mcv', 'tibc', 'Inconclusive di...",3.0,7.0,6.260199,397.645599,...,0.0,-1.000000,48.674998,128.779530,63.832323,21.538620,-1.000000,18.780596,-1.000000,3
171,5.0,171.0,0.0,-1.0,1.0,"['hemoglobin', 'mcv', 'tibc', 'ferritin', 'Inc...",3.0,7.0,9.853701,-1.000000,...,1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,29.561104,-1.000000,3
591,8.0,591.0,0.0,-1.0,1.0,"['hemoglobin', 'mcv', 'tibc', 'gender', 'ferri...",3.0,7.0,9.363798,56.156607,...,1.0,-1.000000,10.257634,106.006058,12.284972,19.218141,-1.000000,28.091394,54.548288,3


In [54]:
acd_hem_pred_df = combined_acd_test_df[(combined_acd_test_df.y_pred==2)]
acd_hem_pred_df.mcv.mean()

nan

In [20]:
#rows with tibc in their trajectory
tibc_acd_df = pd.DataFrame()
count=0
for i, row in acd_test_df.iterrows():
    try:
        traj = ast.literal_eval(row.trajectory)
    except:
        traj = row.trajectory
    if 'tibc' in traj:
        tibc_acd_df = tibc_acd_df.append(row)
tibc_acd_df

,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred
6523,7.0,6523.0,0.0,5.0,0.0,"['mentzer_index', 'rbc', 'iron', 'mcv', 'ferri...",3.0,1.0


In [44]:
X_test_df.loc[6523]

ferritin                   1.774338
ret_count                  0.000000
segmented_neutrophils      0.000000
iron                     115.251457
tibc                     312.645199
rbc                        4.138273
mcv                       74.123819
mentzer_index             17.911776
label                      3.000000
Name: 6523, dtype: float64

#### 2- Aplastic Anemia

In [61]:
aplastic_test_df = test_df[test_df.y_pred == 2]
aplastic_x_test_df = X_test_df[X_test_df.label== 2]

In [63]:
#rows with tibc in their trajectory
neutrophils_aplastic_df = pd.DataFrame()
count=0
for i, row in aplastic_test_df.iterrows():
    try:
        traj = ast.literal_eval(row.trajectory)
    except:
        traj = row.trajectory
    if 'segmented_neutrophils' in traj:
        neutrophils_aplastic_df = neutrophils_aplastic_df.append(row)
neutrophils_aplastic_df

,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred
4590,4.0,4590.0,0.0,2.0,0.0,"['mentzer_index', 'rbc', 'segmented_neutrophil...",4.0,2.0
6627,4.0,6627.0,0.0,2.0,0.0,"['mentzer_index', 'rbc', 'segmented_neutrophil...",4.0,2.0


In [64]:
a2 = aplastic_test_df[aplastic_test_df.y_actual != aplastic_test_df.y_pred]
a2.head()

,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred
2406,4.0,2406.0,0.0,2.0,0.0,"['mentzer_index', 'rbc', 'ret_count', 'Aplasti...",0.0,2.0
4307,4.0,4307.0,0.0,2.0,0.0,"['mentzer_index', 'rbc', 'ret_count', 'Aplasti...",0.0,2.0
4590,4.0,4590.0,0.0,2.0,0.0,"['mentzer_index', 'rbc', 'segmented_neutrophil...",4.0,2.0
6627,4.0,6627.0,0.0,2.0,0.0,"['mentzer_index', 'rbc', 'segmented_neutrophil...",4.0,2.0


In [65]:
a2_index_list = list(a2.index)
a2_x_test_df = X_test_df.iloc[a2_index_list]
a2_x_test_df

,ferritin,ret_count,segmented_neutrophils,iron,tibc,rbc,mcv,mentzer_index,label
2406,0.0,2.005960,0.000000,0.0,0.0,5.229860,90.839971,17.369485,0
4307,0.0,2.034032,0.000000,0.0,0.0,5.631821,99.033033,17.584549,0
4590,0.0,0.000000,0.056872,0.0,0.0,3.808240,102.922809,27.026343,4
6627,0.0,0.000000,0.044406,0.0,0.0,3.971057,104.236046,26.248945,4


In [67]:
X_test_df.describe()

,ferritin,ret_count,segmented_neutrophils,iron,tibc,rbc,mcv,mentzer_index,label
count,8519.000000,8519.000000,8519.000000,8519.000000,8519.000000,8519.000000,8519.000000,8519.000000,8519.000000
mean,206.447490,2.384897,0.107114,6.425037,27.385687,3.517365,89.149873,26.965117,1.097312
std,956.680755,2.252126,0.559969,21.872109,81.948931,0.808308,7.464055,8.035280,1.231690
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.664613,59.569175,12.415871,0.000000
25%,0.000000,0.258182,0.000000,0.000000,0.000000,2.973509,84.166662,21.665629,0.000000
50%,0.000000,1.987357,0.000000,0.000000,0.000000,3.522851,89.346789,25.295998,1.000000
75%,0.000000,3.910286,0.000000,0.000000,0.000000,4.061052,94.316578,30.322435,2.000000
max,10820.137808,12.598495,7.289302,202.702735,573.833319,6.352750,116.700292,129.999157,5.000000


#### 3 - Iron deficiency anemia 

There are a total of 121 IDA test samples. Of these, 3 are diagnosed as hemolytic anemia while 18 are diagnosed as having aplastic anemia 

In [38]:
#ida_test_df = test_df[test_df.y_pred == 3]
ida_test_df = test_df[test_df.y_actual == 3]
ida_x_test_df = X_test_df[X_test_df.label== 3]
combined_ida_test_df = pd.concat([ida_x_test_df, ida_test_df], axis=1)
combined_ida_test_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,label,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred
81,5.204281,4.799012,0.00000,0.0,163.541988,75.774616,3,8.0,81.0,0.0,0.0,1.0,"['hemoglobin', 'mcv', 'ferritin', 'ret_count',...",3.0,NaN
191,9.422081,6.293629,0.00000,0.0,246.411440,77.644708,3,8.0,191.0,0.0,0.0,1.0,"['hemoglobin', 'mcv', 'ferritin', 'ret_count',...",3.0,NaN
206,5.702470,6.713796,0.00000,0.0,242.904336,78.319672,3,8.0,206.0,0.0,0.0,1.0,"['hemoglobin', 'mcv', 'ferritin', 'ret_count',...",3.0,NaN
268,8.741973,5.243427,0.58045,0.0,291.364377,76.053946,3,6.0,268.0,1.0,6.0,0.0,"['hemoglobin', 'mcv', 'ferritin', 'ret_count',...",3.0,3.0
284,11.606463,3.271495,0.00000,0.0,278.757066,76.910957,3,8.0,284.0,0.0,0.0,1.0,"['hemoglobin', 'mcv', 'ferritin', 'ret_count',...",3.0,NaN


In [40]:
combined_ida_test_df.mcv.mean()

75.95405101094063

In [41]:
#Looking at the 3 that are diagnoses as hemolytic
ida_hem_pred_df = combined_ida_test_df[(combined_ida_test_df.y_pred==1)|(combined_ida_test_df.y_pred==2)]
ida_hem_pred_df.mcv.mean()

79.46021283637137

In [25]:
len(ida_hem_pred_df)

118

In [9]:
a3 = ida_test_df[ida_test_df.y_actual != ida_test_df.y_pred]
a3.head()

,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred


In [70]:
#rows with tibc in their trajectory
tibc_ida_df = pd.DataFrame()
count=0
for i, row in ida_test_df.iterrows():
    try:
        traj = ast.literal_eval(row.trajectory)
    except:
        traj = row.trajectory
    if 'tibc' in traj:
        tibc_ida_df = tibc_ida_df.append(row)
tibc_ida_df

,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred
3476,7.0,3476.0,1.0,7.0,0.0,"['mentzer_index', 'rbc', 'iron', 'mcv', 'ferri...",3.0,3.0
3553,7.0,3553.0,1.0,7.0,0.0,"['mentzer_index', 'rbc', 'iron', 'mcv', 'ferri...",3.0,3.0
3956,7.0,3956.0,1.0,7.0,0.0,"['mentzer_index', 'rbc', 'iron', 'mcv', 'ferri...",3.0,3.0
5052,7.0,5052.0,1.0,7.0,0.0,"['mentzer_index', 'rbc', 'iron', 'mcv', 'ferri...",3.0,3.0


In [71]:
b3_index_list = list(tibc_ida_df.index)
b3_x_test_df = X_test_df.iloc[b3_index_list]
b3_x_test_df

,ferritin,ret_count,segmented_neutrophils,iron,tibc,rbc,mcv,mentzer_index,label
3476,1.882835,0.0,0.0,151.140276,309.885395,2.767909,78.672609,28.423117,3
3553,1.836332,0.0,0.0,151.640269,113.897048,3.129228,74.955135,23.953234,3
3956,1.737838,0.0,0.0,113.872103,211.983293,1.190145,78.387423,65.863742,3
5052,1.765401,0.0,0.0,20.020334,298.601157,1.939750,77.485236,39.945995,3
